##### Please do not run the section2 for the hyperparameter tuning, only run the section1 and section3 to import requirments and the experiments.

# 1. Requirements

In [1]:
import anchor_estimator
import importlib
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from helper import (
    load_dataset,
    split_data,
    run_single_experiment,
    run_all_experiments,
    tune_hyperparams
)

from cnn_model import cnn_model
from loss_functions import (
    symmetric_cross_entropy,
    forward_correction_loss,
    CoTeachingProxyLoss,
    RememberRateScheduler,
    _infer_noise_rate_from_name, 
)

/Users/nir/Documents/aml/aml-tm/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# 2. Hyper-parameter tuning code: do not run

In [ ]:
# datasets = ['FashionMNIST0.3', 'FashionMNIST0.6', 'CIFAR']
# methods  = ['sce']

# # One-time tuning per dataset × method
# tuned = {}  # key: (dataset, method) -> param dict
# for dataset in datasets:
#     data_path = f'datasets/{dataset}.npz'
#     Xtr, Str, Xts, Yts, T = load_dataset(data_path, dataset)
#     input_shape = Xtr.shape[1:]

#     for method in methods:
#         print(f"\n=== Tuning {method.upper()} on {dataset} ===")
#         best_params = tune_hyperparams(
#             Xtr, Str, dataset, method, input_shape,
#             n_dev_runs=3, epochs=30  # lightweight dev budget
#         )
#         tuned[(dataset, method)] = best_params


=== Tuning SCE on FashionMNIST0.3 ===


2025-11-05 21:46:45.502965: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4 Max
2025-11-05 21:46:45.502997: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 36.00 GB
2025-11-05 21:46:45.503003: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 13.50 GB
I0000 00:00:1762339605.503394  316267 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1762339605.503442  316267 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2025-11-05 21:46:45.960863: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


KeyboardInterrupt: 

## 2.1 Hyperparams for sce

In [ ]:
# hyperparams for sce
tuned

{('FashionMNIST0.3', 'sce'): {'alpha': 0.01,
  'beta': 1.0,
  'A': -1.0,
  'lr': 0.001},
 ('FashionMNIST0.6', 'sce'): {'alpha': 0.01,
  'beta': 0.5,
  'A': -4.0,
  'lr': 0.001},
 ('CIFAR', 'sce'): {'alpha': 0.05, 'beta': 1.0, 'A': -4.0, 'lr': 0.001}}

## 2.2 Hyperparams for COTEACHING

In [ ]:
# datasets = ['FashionMNIST0.3', 'FashionMNIST0.6', 'CIFAR']
# methods  = ['coteaching']

# # One-time tuning per dataset × method
# tuned_coteaching = {}  # key: (dataset, method) -> param dict
# for dataset in datasets:
#     data_path = f'datasets/{dataset}.npz'
#     Xtr, Str, Xts, Yts, T = load_dataset(data_path, dataset)
#     input_shape = Xtr.shape[1:]

#     for method in methods:
#         print(f"\n=== Tuning {method.upper()} on {dataset} ===")
#         best_params = tune_hyperparams(
#             Xtr, Str, dataset, method, input_shape,
#             n_dev_runs=3, epochs=30  # lightweight dev budget
#         )
#         tuned_coteaching[(dataset, method)] = best_params


=== Tuning COTEACHING on FashionMNIST0.3 ===
[TUNE] dataset=FashionMNIST0.3 method=coteaching params={'warmup': 3, 'lr': 0.001} -> val_acc=0.6779
[TUNE] dataset=FashionMNIST0.3 method=coteaching params={'warmup': 3, 'lr': 0.0005} -> val_acc=0.6778
[TUNE] dataset=FashionMNIST0.3 method=coteaching params={'warmup': 5, 'lr': 0.001} -> val_acc=0.6801
[TUNE] dataset=FashionMNIST0.3 method=coteaching params={'warmup': 5, 'lr': 0.0005} -> val_acc=0.6784
[TUNE] dataset=FashionMNIST0.3 method=coteaching params={'warmup': 10, 'lr': 0.001} -> val_acc=0.6811
[TUNE] dataset=FashionMNIST0.3 method=coteaching params={'warmup': 10, 'lr': 0.0005} -> val_acc=0.6756
[TUNE][BEST] dataset=FashionMNIST0.3 method=coteaching -> {'warmup': 10, 'lr': 0.001} (val_acc=0.6811)

=== Tuning COTEACHING on FashionMNIST0.6 ===
[TUNE] dataset=FashionMNIST0.6 method=coteaching params={'warmup': 3, 'lr': 0.001} -> val_acc=0.3821
[TUNE] dataset=FashionMNIST0.6 method=coteaching params={'warmup': 3, 'lr': 0.0005} -> val_ac

In [ ]:
tuned_coteaching

{('FashionMNIST0.3', 'coteaching'): {'warmup': 10, 'lr': 0.001},
 ('FashionMNIST0.6', 'coteaching'): {'warmup': 5, 'lr': 0.0005},
 ('CIFAR', 'coteaching'): {'warmup': 5, 'lr': 0.0005}}

# 3. Experiments

In [2]:
datasets = ['FashionMNIST0.3', 'FashionMNIST0.6', 'CIFAR']
methods = ['baseline','forward', 'sce', 'coteaching'] 

result = run_all_experiments(datasets, methods, 10, 50)

Running BASELINE on FashionMNIST0.3...


2025-11-06 13:46:44.735144: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4 Max
2025-11-06 13:46:44.735180: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 36.00 GB
2025-11-06 13:46:44.735187: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 13.50 GB
I0000 00:00:1762397204.735226   45579 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1762397204.735269   45579 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2025-11-06 13:46:45.105909: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


Run 1/10: Test Accuracy = 93.03%
Run 2/10: Test Accuracy = 94.93%
Run 3/10: Test Accuracy = 91.27%
Run 4/10: Test Accuracy = 97.80%
Run 5/10: Test Accuracy = 94.33%
Run 6/10: Test Accuracy = 92.40%
Run 7/10: Test Accuracy = 92.93%
Run 8/10: Test Accuracy = 95.13%
Run 9/10: Test Accuracy = 95.77%
Run 10/10: Test Accuracy = 91.03%
Result: 93.86 ± 2.01%
Running FORWARD on FashionMNIST0.3...
Run 1/10: Test Accuracy = 98.97%
Run 2/10: Test Accuracy = 99.10%
Run 3/10: Test Accuracy = 98.83%
Run 4/10: Test Accuracy = 99.17%
Run 5/10: Test Accuracy = 99.23%
Run 6/10: Test Accuracy = 99.00%
Run 7/10: Test Accuracy = 98.90%
Run 8/10: Test Accuracy = 99.17%
Run 9/10: Test Accuracy = 99.13%
Run 10/10: Test Accuracy = 99.20%
Result: 99.07 ± 0.13%
Running SCE on FashionMNIST0.3...
Run 1/10: Test Accuracy = 98.63%
Run 2/10: Test Accuracy = 97.53%
Run 3/10: Test Accuracy = 98.43%
Run 4/10: Test Accuracy = 98.30%
Run 5/10: Test Accuracy = 98.57%
Run 6/10: Test Accuracy = 98.80%
Run 7/10: Test Accuracy 

In [3]:
result.to_csv('result.csv')

In [4]:
result = pd.read_csv('result.csv')
pivot_df = result.pivot(index='Dataset', columns='Method', values='Result')
    
print(pivot_df)

Method               BASELINE    COTEACHING       FORWARD           SCE
Dataset                                                                
CIFAR            61.19 ± 3.58  59.56 ± 6.45  68.66 ± 2.03  64.39 ± 2.53
FashionMNIST0.3  93.86 ± 2.01  97.52 ± 1.72  99.07 ± 0.13  98.19 ± 0.77
FashionMNIST0.6  94.87 ± 0.95  90.94 ± 3.30  91.59 ± 9.74  95.51 ± 0.60
